In [1]:
# Team 1 LendingClub primary notebook
# import dependencies
import pandas as pd
import numpy as np  # needed for replace nan code below
import glob

# PROCESSING OF LENDING CLUB LOAN FILES

# pull in all csvs and merge them into a single dataframe
bigdf = pd.concat([pd.read_csv(f, low_memory=False) for f in
                 glob.glob('data/LoanStats*.csv')], ignore_index=True)

# drop irrelevant rows
data = bigdf.drop([
                "id", "member_id", "url", "desc", "hardship_type", "hardship_reason",
                "hardship_status", "deferral_term", "hardship_amount",
                "hardship_start_date", "hardship_end_date", "payment_plan_start_date",
                "hardship_length", "hardship_dpd", "hardship_loan_status",
                "orig_projected_additional_accrued_interest",
                "hardship_payoff_balance_amount", "hardship_last_payment_amount",
                "debt_settlement_flag_date", "settlement_status", "settlement_date",
                "loan_amnt", "funded_amnt", "grade", "sub_grade",
                "emp_title", "verification_status", "dti", "earliest_cr_line",
                "inq_last_6mths", "zip_code", "pub_rec", "revol_bal",
                "total_acc", "initial_list_status", "out_prncp", "out_prncp_inv",
                "total_pymnt", "total_pymnt_inv", "total_rec_prncp", "total_rec_int",
                "last_pymnt_d", "last_pymnt_amnt", "next_pymnt_d", "last_credit_pull_d",
                "collections_12_mths_ex_med", "policy_code", "application_type",
                "annual_inc_joint", "dti_joint", "verification_status_joint",
                "acc_now_delinq", "tot_coll_amt", "tot_cur_bal", "open_acc_6m",
                "open_act_il", "open_il_12m", "open_il_24m", "mths_since_rcnt_il",
                "total_bal_il", "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
                "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl", "inq_last_12m",
                "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy", "bc_util",
                "num_accts_ever_120_pd", "num_actv_bc_tl", "num_actv_rev_tl",
                "num_bc_sats", "num_bc_tl", "num_il_tl", "num_op_rev_tl",
                "num_rev_accts", "num_rev_tl_bal_gt_0", "num_sats",
                "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
                "num_tl_op_past_12m", "percent_bc_gt_75", "pub_rec_bankruptcies",
                "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
                "total_il_high_credit_limit", "revol_bal_joint",
                "sec_app_earliest_cr_line", "sec_app_inq_last_6mths", "sec_app_mort_acc",
                "sec_app_open_acc", "sec_app_revol_util", "sec_app_num_rev_accts",
                "sec_app_chargeoff_within_12_mths", "sec_app_collections_12_mths_ex_med",
                "sec_app_mths_since_last_major_derog", "hardship_flag",
                "disbursement_method", "debt_settlement_flag",
                "debt_settlement_flag_date", "settlement_status", "settlement_date",
                "settlement_amount", "settlement_percentage", "settlement_term"], axis=1)

# code to fill nans with 0s
data['mths_since_last_major_derog'].replace(np.nan, 0)

# processing date into new columns
data["month_num"] = ""
data.loc[data['issue_d'].str[:3] == "Dec", ('month_num')] = (12)
data.loc[data['issue_d'].str[:3] == "Nov", ('month_num')] = (11)
data.loc[data['issue_d'].str[:3] == "Oct", ('month_num')] = (10)
data.loc[data['issue_d'].str[:3] == "Sep", ('month_num')] = (9)
data.loc[data['issue_d'].str[:3] == "Aug", ('month_num')] = (8)
data.loc[data['issue_d'].str[:3] == "Jul", ('month_num')] = (7)
data.loc[data['issue_d'].str[:3] == "Jun", ('month_num')] = (6)
data.loc[data['issue_d'].str[:3] == "May", ('month_num')] = (5)
data.loc[data['issue_d'].str[:3] == "Apr", ('month_num')] = (4)
data.loc[data['issue_d'].str[:3] == "Mar", ('month_num')] = (3)
data.loc[data['issue_d'].str[:3] == "Feb", ('month_num')] = (2)
data.loc[data['issue_d'].str[:3] == "Jan", ('month_num')] = (1)
data.loc[data['issue_d'].str[:3] == "", ('month_num')] = (0)

# clean up the dead rows
data.dropna(subset=['term', 'issue_d'], inplace=True)

# PROCESSING OF FEMA DISASTER FILE

# read in FEMA csv file into dataframe
fema = pd.read_csv('data/DisasterDeclarationsSummaries.csv')

# drop columns that are irrelevant
fema = fema.loc[:, ['disasterNumber', 'state', 'incidentBeginDate',
                'incidentType', 'incidentEndDate']]

# create columns to link with lendingclub.com data
fema['yearBegin'] = fema['incidentBeginDate'].str[:4]
fema['monthBegin'] = fema['incidentBeginDate'].str[5:7]
fema['monthEnd'] = fema['incidentEndDate'].str[5:7]

# drop unix dates from dataframe
fema = fema.loc[:, ['disasterNumber', 'state', 'incidentType', 'yearBegin',
                'monthBegin', 'monthEnd']]

# drop all data outside of 2018
fema = fema[fema['yearBegin'] == '2018']

# drop duplicate rows with same incident named for multiple counties
fema = fema.drop_duplicates(subset=['disasterNumber'], keep=False)

# fill in all ending month with beginning month if it is missing
fema.monthEnd.fillna(fema.monthBegin, inplace=True)

# drop irrelevant disasterNumber
fema.drop(columns=['disasterNumber'], inplace=True)

# convert month number strings to numeric
fema['monthBegin'] = fema['monthBegin'].apply(pd.to_numeric, errors='coerce')
fema['monthEnd'] = fema['monthEnd'].apply(pd.to_numeric, errors='coerce')
fema['aftermath'] = fema['monthEnd'] + 1

# reinforce the numeric status of loan month number
data['month_num'] = data['month_num'].apply(pd.to_numeric, errors='coerce')

# clean up the dead rows
fema.dropna(subset=['monthBegin', 'monthEnd'], inplace=True)
# write to csv
# fema.to_csv('data/fema_clean.csv', index=False)

# CREATING TRIGGER MARKS IN LENDING CLUB BASED ON FEMA

data["Disaster"] = ""

# newdf = pd.merge(data, fema, how='left'. left_on=['month_num','addr_state'], right_on=['aftermath','state'])
i = 0


#for index1, row1 in data.iterrows():
#    data.at[index1,"Disaster"] = "D"
#    i = i + 1
#    if i == 5:
#        break

# print(data.head())
# for index1, row1 in data.iterrows():
#     i = i + 1
#     print(f"Processing row: {i}")
#     if i == len(data.index):
#         break
#     for index2, row2 in fema.iterrows():
#         if (row1['month_num'] == row2['monthEnd'] + 1 and row1['addr_state'] == row2['state']):
#             data.at[index1,"Disaster"] = "D"
#             print("Yes")
    #    else:
    #        row1["Disaster"] = "N"
    #        print("No")

# data.to_csv('data/data_fema_compare.csv', index=False)


In [5]:
def checkDisaster(row):
    global fema
    tmp_df = fema[(fema['monthEnd']+1==row['month_num']) & (fema['state']==row['addr_state'])]
    if len(tmp_df)>0:
        return 'D'
    else:
        return 'N'
    
data['Disaster'] = data.apply(lambda x: checkDisaster(x), axis=1)

In [6]:
data.head()


funded_amnt_inv        term int_rate  installment emp_length  \
0          30000.0   36 months    7.34%       930.99    3 years   
1          34825.0   60 months   12.61%       785.45    2 years   
2           2600.0   36 months    7.96%        81.43    3 years   
3          20000.0   60 months    9.92%       424.16    3 years   
4          17000.0   60 months   20.39%       454.10  10+ years   

  home_ownership  annual_inc issue_d loan_status pymnt_plan    ...     \
0            OWN     95000.0  Mar-18  Fully Paid          n    ...      
1       MORTGAGE    125000.0  Mar-18     Current          n    ...      
2       MORTGAGE     62000.0  Mar-18     Current          n    ...      
3       MORTGAGE    110000.0  Mar-18     Current          n    ...      
4           RENT     52000.0  Mar-18     Current          n    ...      

  mo_sin_rcnt_tl mort_acc mths_since_recent_bc  mths_since_recent_bc_dlq  \
0           13.0      0.0                 19.0                       NaN   
1           14.0      1.0                 14.0                      45.0   
2           19.0      1.0                 45.0                       NaN   
3            3.0      5.0                  8.0                       NaN   
4           19.0      0.0                 19.0                      77.0   

   mths_since_recent_inq  mths_since_recent_revol_delinq  pct_tl_nvr_dlq  \
0                   24.0                            10.0            90.0   
1                   15.0                            45.0            96.3   
2                    NaN                             NaN           100.0   
3                    4.0                             NaN            90.9   
4                    0.0                            77.0            85.7   

  sec_app_open_act_il  month_num  Disaster  
0                 NaN          3         N  
1                 NaN          3         N  
2                 NaN          3         N  
3                 NaN          3         N  
4                 NaN          3         N  

[5 rows x 37 columns]

In [3]:
fema.head()

state incidentType yearBegin  monthBegin  monthEnd  aftermath
48064    OK         Fire      2018           3         3          4
48126    OK         Fire      2018           4         4          5
48127    OK         Fire      2018           4         4          5
48128    TX         Fire      2018           4         4          5
48129    TX         Fire      2018           4         4          5